In [ ]:
!pip install torchtext==0.15.1
!pip install datasets

In [ ]:
from datasets import load_dataset

ds2016 = load_dataset("turuta/Multi30k-uk", "flickr_2016")
ds2017 = load_dataset("turuta/Multi30k-uk", "flickr_2017")
ds2018 = load_dataset("turuta/Multi30k-uk", "flickr_2018")
multi = load_dataset("turuta/Multi30k-uk", "multi30k")

In [ ]:
print(f'ds2016 {ds2016}')
print(f"ds2017 {ds2017}")
print(f"ds2018 {ds2018}")
print(f"multi {multi}")

In [ ]:
from datasets import concatenate_datasets, DatasetDict
from sklearn.model_selection import train_test_split as sklearn_train_test_split

combined_train = concatenate_datasets([
    ds2016["train"],
    ds2017["train"],
    ds2018["train"],
    multi["train"]
])

train_indices, test_indices = sklearn_train_test_split(
    list(range(len(combined_train))),
    test_size=0.2,
    random_state=42
)

train_dataset = combined_train.select(train_indices)
test_dataset = combined_train.select(test_indices)

combined_dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(combined_dataset)

In [ ]:
!python -m spacy download uk_core_news_sm
!python -m spacy download en_core_web_sm

In [ ]:
import spacy

uk_nlp = spacy.load("uk_core_news_sm")
en_nlp = spacy.load("en_core_web_sm")

In [ ]:
import re

def clean_tokens(tokens):
    return [
        token for token in tokens
        if token.strip() and token.strip() not in {",", ".", "!", "?", ":", ";", "\xa0", " "}
    ]

def tokenize_example(example, uk_nlp, en_nlp, max_length, lower, sos_token, eos_token):
    uk_tokens = [token.text for token in uk_nlp.tokenizer(example["uk"])][:max_length]
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    if lower:
        uk_tokens = [token.lower() for token in uk_tokens]
        en_tokens = [token.lower() for token in en_tokens]

    uk_tokens = clean_tokens(uk_tokens)
    en_tokens = clean_tokens(en_tokens)

    uk_tokens = [sos_token] + uk_tokens + [eos_token]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    return {"uk_tokens": uk_tokens, "en_tokens": en_tokens}

In [ ]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "uk_nlp": uk_nlp,
    "en_nlp": en_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_dataset.map(tokenize_example, fn_kwargs=fn_kwargs)

In [ ]:
train_data[0]

In [ ]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

uk_vocab = build_vocab_from_iterator(
    train_data["uk_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

en_vocab = build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [ ]:
uk_vocab.get_itos()[:20]

In [ ]:
en_vocab.get_itos()[:20]

In [ ]:
len(uk_vocab), len(en_vocab)

In [ ]:
assert uk_vocab[unk_token] == en_vocab[unk_token]
assert uk_vocab[pad_token] == en_vocab[pad_token]

unk_index = uk_vocab[unk_token]
pad_index = uk_vocab[pad_token]

In [ ]:
uk_vocab.set_default_index(unk_index)
en_vocab.set_default_index(unk_index)

In [ ]:
tokens = ['a',
 'in',
 'the',
 'on',
 'man',
 'is',
 'and',
 'of']
en_vocab.lookup_indices(tokens)

In [ ]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

In [ ]:
def numericalize_example(example, uk_vocab, en_vocab):
    uk_ids = uk_vocab.lookup_indices(example["uk_tokens"])
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    return {"uk_ids": uk_ids, "en_ids": en_ids}

In [ ]:
fn_kwargs = {"uk_vocab": uk_vocab, "en_vocab": en_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

In [ ]:
train_data[0]

In [ ]:
uk_vocab.lookup_tokens(train_data[0]["uk_ids"])

In [ ]:
data_type = "torch"
format_columns = ["uk_ids", "en_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

In [ ]:
train_data[0]

In [ ]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_uk_ids = [example["uk_ids"] for example in batch]
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_uk_ids = nn.utils.rnn.pad_sequence(batch_uk_ids, padding_value=pad_index)
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch = {
            "uk_ids": batch_uk_ids,
            "en_ids": batch_en_ids,
        }
        return batch

    return collate_fn

In [ ]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [ ]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

In [ ]:
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]
        # input = [batch size]
        for t in range(1, trg_length):
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            # input = [batch size]
        return outputs

In [ ]:
input_dim = len(uk_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [ ]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["uk_ids"].to(device)
        trg = batch["en_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
from tqdm import tqdm
import numpy as np

n_epochs = 20
clip = 1.0
teacher_forcing_ratio = 0.5

for epoch in tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )

    print(f"Epoch {epoch + 1}/{n_epochs}")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json

model_save_path = '/content/drive/MyDrive/lstm/my_model_uk-en.pth'
torch.save(model.state_dict(), model_save_path)

vocab_save_path = '/content/drive/MyDrive/lstm/vocab.json'

with open(vocab_save_path, 'w') as f:
    json.dump({
        "uk_vocab": uk_vocab.get_stoi(),
        "en_vocab": en_vocab.get_stoi()
    }, f)


# Новий розділ

In [ ]:
def translate_sentence(
    sentence,
    model,
    uk_nlp,
    en_nlp,
    uk_vocab,
    en_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in uk_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]

        if lower:
            tokens = [token.lower() for token in tokens]

        tokens = [sos_token] + tokens + [eos_token]
        ids = uk_vocab.lookup_indices(tokens)

        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)

        hidden, cell = model.encoder(tensor)

        inputs = en_vocab.lookup_indices([sos_token])

        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)

            if predicted_token == en_vocab[eos_token]:
                break

        tokens = en_vocab.lookup_tokens(inputs)

    return tokens

In [ ]:
sentence = test_data[0]["uk"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

In [ ]:
translation = translate_sentence(
    sentence,
    model,
    uk_nlp,
    en_nlp,
    uk_vocab,
    en_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [ ]:
translation

In [ ]:
translations = [
    translate_sentence(
        example["uk"],
        model,
        uk_nlp,
        en_nlp,
        uk_vocab,
        en_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm(test_data)
]

In [ ]:
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["en"]] for example in test_data]

In [ ]:
predictions[0], references[0]

In [ ]:
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

In [ ]:
tokenizer_fn = get_tokenizer_fn(en_nlp, lower)

In [ ]:
predictions = [tokenizer_fn(" ".join(translation[1:-1])) for translation in translations]
references = [[tokenizer_fn(example["en"].strip())] for example in test_data]

In [ ]:
from torchtext.data.metrics import bleu_score

score = bleu_score(
    predictions, references
)

print(f"BLEU score: {score}")